# Image Classification
This notebook deals with the image classification portion of the final project. We'll be utilizing a Convolutional Neural Network from the Tensorflow framework.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import image
from IPython.display import clear_output
import os
import shutil
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
len(tf.config.list_physical_devices('GPU'))

0

## Prepping our data
We'll need split our images into train and test sets. To do this we'll create two new directories, train_images and test_images. We'll make a function to make theses directories.

In [3]:
def makeFolder(subfolder):
    parent_dir = os.getcwd()
    path = os.path.join(parent_dir,subfolder)
    try:
        os.mkdir(path)
        print('Successfully made directory.')
    except OSError as error:
        print(error)

Now we'll make the directories.

In [4]:
makeFolder('train_images')
makeFolder('test_images')

[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\sulli\\Code\\Python\\Data Mining\\Final Project\\train_images'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\sulli\\Code\\Python\\Data Mining\\Final Project\\test_images'


We'll create a function to copy to the directory.

In [5]:
def copyDir(destination):
    current = os.getcwd()
    for sub in ['animal_crossing','doom']:
        try:
            shutil.copytree(f'{current}\\{sub}',f'{current}\\{destination}\\{sub}')
            print(f'Successfully copied {sub} to {destination}')
        except OSError as error:
            print(error)

Now we'll copy to our directory.

In [6]:
copyDir('train_images')
copyDir('test_images')

[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\sulli\\Code\\Python\\Data Mining\\Final Project\\train_images\\animal_crossing'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\sulli\\Code\\Python\\Data Mining\\Final Project\\train_images\\doom'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\sulli\\Code\\Python\\Data Mining\\Final Project\\test_images\\animal_crossing'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\sulli\\Code\\Python\\Data Mining\\Final Project\\test_images\\doom'


Next we get our image file names for each.

In [7]:
train = pd.read_csv('train.csv')['filename']
test = pd.read_csv('test.csv')['filename']

Now we'll loop through our directories and remove the files not in the specified list.

In [8]:
def removeFilesNotInList(array,folder):
    flist = array.tolist()
    for sub in ['animal_crossing','doom']:
        subpath = os.path.join(folder,sub)
        for file in os.listdir(subpath):
            f = os.path.join(subpath,file)
            if file not in flist:
                os.remove(f)

In [9]:
removeFilesNotInList(train,'.\\train_images')
removeFilesNotInList(test,'.\\test_images')

Now let's load our training images into tensorflow.

In [10]:
train_ds = image_dataset_from_directory('.\\train_images',image_size=(180,180),
                                       batch_size=32)

Found 1276 files belonging to 2 classes.


And now our testing dataset

In [11]:
test_ds = image_dataset_from_directory('.\\test_images',image_size=(180,180),
                                       batch_size=32)

Found 318 files belonging to 2 classes.


Let's assign our class names 

In [12]:
class_names = train_ds.class_names

Now let's create our model

In [13]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [14]:
model = models.Sequential()
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

In [15]:
model.add(layers.Flatten())
model.add(layers.Dense(32,activation='softmax'))
model.add(layers.Dense(2))

In [16]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])
history =  model.fit(train_ds,epochs=5)

Epoch 1/5
40/40 [==============================] - 14s 314ms/step - loss: 0.6675 - accuracy: 0.6176
Epoch 2/5
40/40 [==============================] - 14s 328ms/step - loss: 0.6339 - accuracy: 0.6951
Epoch 3/5
40/40 [==============================] - 14s 325ms/step - loss: 0.6041 - accuracy: 0.7328
Epoch 4/5
40/40 [==============================] - 14s 329ms/step - loss: 0.5828 - accuracy: 0.7539
Epoch 5/5
40/40 [==============================] - 14s 325ms/step - loss: 0.5876 - accuracy: 0.7304


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 19, 19, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 32)        1

In [18]:
predictions = model.predict(test_ds)

10/10 [==============================] - 2s 72ms/step


In [19]:
model.evaluate(test_ds)

10/10 [==============================] - 2s 70ms/step - loss: 0.5591 - accuracy: 0.7642


[0.5590914487838745, 0.7641509175300598]

In [20]:
predictions

array([[-0.15519017,  0.49576184],
       [ 0.02938347,  0.18104464],
       [-0.15437633,  0.49372667],
       [ 0.33257258, -0.32427335],
       [-0.1494257 ,  0.48085812],
       [-0.15519089,  0.4957601 ],
       [ 0.39517793, -0.42571092],
       [-0.15519017,  0.49576184],
       [-0.1557341 ,  0.49413642],
       [-0.15526241,  0.49557814],
       [-0.1552152 ,  0.4957047 ],
       [-0.15519017,  0.49576184],
       [ 0.1703845 , -0.0540685 ],
       [ 0.39663902, -0.42760956],
       [-0.06593987,  0.33815762],
       [-0.15364523,  0.49153283],
       [ 0.39417115, -0.42441857],
       [-0.15407479,  0.49310717],
       [-0.15520927,  0.4953623 ],
       [-0.15519068,  0.4957609 ],
       [-0.15519029,  0.49576157],
       [-0.15519027,  0.4957616 ],
       [-0.08232725,  0.3044012 ],
       [ 0.39440098, -0.42475563],
       [ 0.22428319, -0.14177024],
       [-0.15519017,  0.49576184],
       [-0.15519017,  0.49576184],
       [ 0.39534009, -0.425946  ],
       [ 0.39348307,

In [21]:
predicted_class = np.amax(predictions)

In [22]:
len(predictions)

318

In [23]:
test_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>